In [6]:
import scipy
from scipy.optimize import OptimizeResult
%load_ext autoreload
%autoreload 2

from core.optimizer_evaluator import *
from core.abstract_analzyer import *
import scipy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Сравнение `scipy.optimize.minimize` и `scipy.optimize.least_squares` с нашими методами

In [2]:
def loop(_):
    while True:
        pass
infinite = Algorithm("infinite", loop)
def err(_):
    raise Exception
error = Algorithm("error", err)
constantly = Algorithm("constantly", lambda _: "Hello")

compare_optimization_algorithms_in_table([infinite, error, constantly], [Problem("1", lambda x: x, [], lambda x: 1, None), Problem("2", lambda x: x, [], lambda x: 1, None)], 1)

=========== Testing infinite ===========
Got result TL at problem=1
Got result TL at problem=2
=========== Testing error ===========
Exception occurred: 
Got result × at problem=1
Exception occurred: 
Got result × at problem=2
=========== Testing constantly ===========
Got result Hello at problem=1
Got result Hello at problem=2
+-----------+------------+---------+--------------+
|   problem | infinite   | error   | constantly   |
+===========+============+=========+==============+
|         1 | TL         | ×       | Hello        |
+-----------+------------+---------+--------------+
|         2 | TL         | ×       | Hello        |
+-----------+------------+---------+--------------+


In [4]:
from scipy.optimize import HessianUpdateStrategy


def scipy_optimize_with_solver(solver_name: str, result_extractor, **kwargs):
    def solve(problem: Problem):
        opt_res = scipy.optimize.minimize(
            problem.function,
            problem.x0,
            method=solver_name,
            jac=problem.gradient,
            hess = problem.hessian,
            **kwargs
        )
        return result_extractor(opt_res)

    return Algorithm(solver_name, solve)

In [7]:
def point_number_extractor(opt_res: OptimizeResult):
    return opt_res.nit